In [1]:
import s1_prepare_data as s1
import pandas as pd

'input current date: '

20190927


In [2]:
PD = s1.Prepare_Data()

In [3]:
PD.login_and_choose_database(database = 'python_clickhouse_test')

In [4]:
PD.raw_query('show tables;')

[('account_table',),
 ('basic_table',),
 ('basic_table_sample',),
 ('fraud_case_table',),
 ('information_table',)]

## create table

In [5]:
PD.create_table(table_name = 'basic_table',
             columns = 'cust_id String, date Date',
             engine = 'MergeTree',
             ordered_column = 'cust_id')

query overview:
CREATE TABLE IF NOT EXISTS basic_table
 (cust_id String, date Date)
 ENGINE = MergeTree
 ORDER BY cust_id

show tables:
[('basic_table',)]


In [6]:
PD.create_table(table_name = 'fraud_case_table',
             columns = 'cust_id String, date Date, fraud_label Int64, risk_level Int64',
             engine = 'MergeTree',
             ordered_column = 'cust_id')

query overview:
CREATE TABLE IF NOT EXISTS fraud_case_table
 (cust_id String, date Date, fraud_label Int64, risk_level Int64)
 ENGINE = MergeTree
 ORDER BY cust_id

show tables:
[('basic_table',), ('fraud_case_table',)]


In [7]:
PD.create_table(table_name = 'account_table',
             columns = 'cust_id String, date Date, amount Int64',
             engine = 'MergeTree',
             ordered_column = 'cust_id')

query overview:
CREATE TABLE IF NOT EXISTS account_table
 (cust_id String, date Date, amount Int64)
 ENGINE = MergeTree
 ORDER BY cust_id

show tables:
[('account_table',), ('basic_table',), ('fraud_case_table',)]


In [8]:
PD.create_table(table_name = 'information_table',
             columns = 'cust_id String, date Date, gender String, job String',
             engine = 'MergeTree',
             ordered_column = 'cust_id')

query overview:
CREATE TABLE IF NOT EXISTS information_table
 (cust_id String, date Date, gender String, job String)
 ENGINE = MergeTree
 ORDER BY cust_id

show tables:
[('account_table',), ('basic_table',), ('fraud_case_table',), ('information_table',)]


## insert data

In [9]:
!cat 'fake_data/basic_table.csv' | clickhouse-client -d python_clickhouse_test --query='insert into basic_table format CSV' \
--input_format_allow_errors_ratio=0.01

In [10]:
!cat 'fake_data/fraud_case_table.csv' | clickhouse-client -d python_clickhouse_test --query='insert into fraud_case_table format CSV' \
--input_format_allow_errors_ratio=0.01

In [11]:
!cat 'fake_data/account_table.csv' | clickhouse-client -d python_clickhouse_test --query='insert into account_table format CSV' \
--input_format_allow_errors_ratio=0.1

In [12]:
!cat 'fake_data/information_table.csv' | clickhouse-client -d python_clickhouse_test --query='insert into information_table format CSV' \
--input_format_allow_errors_ratio=0.1

In [13]:
PD.raw_query('select count() from basic_table'),\
PD.raw_query('select count() from fraud_case_table'),\
PD.raw_query('select count() from account_table'),\
PD.raw_query('select count() from information_table')

([(500000,)], [(500000,)], [(5000000,)], [(500000,)])

In [14]:
pd.DataFrame(PD.raw_query('select * from information_table limit 5'),columns=['cust_id','date','gender','job'])

,cust_id,date,gender,job
0,2000007878,2018-12-19,Male,~
1,2000008555,2018-11-01,Female,Engineer
2,2000024405,2018-05-28,Male,Teacher
3,2000031035,2018-08-10,~,Students
4,2000031841,2018-11-05,Female,Accounter


In [5]:
PD.get_dataframe_head(table_name = 'information_table', columns = ['cust_id','date','gender','job'])

,cust_id,date,gender,job
0,2000007878,2018-12-19,Male,~
1,2000008555,2018-11-01,Female,Engineer
2,2000024405,2018-05-28,Male,Teacher
3,2000031035,2018-08-10,~,Students
4,2000031841,2018-11-05,Female,Accounter


## sample data

In [6]:
PD.get_random_sample(current_table = 'basic_table',
                    new_table = 'basic_table_sample',
                    num_cases = '10',
                    ordered_column = 'cust_id',
                    selected_columns = '*')

query overview:
CREATE TABLE IF NOT EXISTS basic_table_sample
 ENGINE = MergeTree
 ORDER BY cust_id
 AS SELECT *
 FROM basic_table
 ORDER BY rand() 
 LIMIT 10

show tables:
[('account_table',), ('basic_table',), ('basic_table_sample',), ('fraud_case_table',), ('information_table',)]


In [8]:
PD.get_dataframe_head(table_name = 'basic_table_sample', columns = ['cust_id','date'])

,cust_id,date
0,2100381844,2018-02-19
1,2208761863,2018-10-13
2,2282664615,2018-08-10
3,2712683577,2018-11-02
4,2738636012,2018-03-12


In [9]:
PD.raw_query('desc basic_table_sample;')

[('cust_id', 'String', '', '', '', ''), ('date', 'Date', '', '', '', '')]

In [10]:
PD.raw_query('select count() from basic_table_sample;')

[(10,)]